In [2]:
# BASIC LIBS

# data
import pandas as pd
import numpy as np

import geopandas as gpd
import contextily as ctx
from pyproj import CRS

# stat significance
import scipy as sp

# visualisation
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()
%config InlineBackend.figure_format = 'retina'


# JUPYTER CONFIG
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

# Data

## TRIPS

### load, format, hbw+hbo

In [3]:
df = pd.read_csv('./data/georgia_od_data/GDOT_2019_09_large.csv')

In [211]:
df_sample = df

In [212]:
# df.sample(10000, random_state = 42).total_linked_trips.describe()

In [213]:
# SAMPLE_SIZE = 100

# # Step 1: Sample 100 unique ORIG values (or as many as exist if less than 100)
# sample_size = min(SAMPLE_SIZE, len(df))  # Handle cases where df has fewer than 100 rows
# sample = df["origin_zone_id"].sample(sample_size, random_state=42).to_list()  # Random sample with reproducibility

# # Step 2: Extract rows where ORIG or DEST matches any value in the sample
# # Convert sample to a set for efficient membership checking
# sample_set = set(sample)

# # Query rows where ORIG or DEST matches sample values
# # df_sample = df[(df["origin_zone_id"].isin(sample_set)) | (df["destination_zone_id"].isin(sample_set))]
# df_sample = df[(df["origin_zone_id"].isin(sample_set))]
# df_sample.shape

In [214]:
df_sample.columns[1:20]

Index(['month', 'origin_zone_id', 'destination_zone_id', 'origin_zone_name',
       'destination_zone_name', 'origin_state', 'destination_state',
       'origin_flag', 'destination_flag', 'day_type', 'avg_linked_trips',
       'avg_unlinked_trips', 'avg_tours', 'total_linked_trips',
       'total_unlinked_trips', 'total_tours', 'hour_00', 'hour_01', 'hour_02'],
      dtype='object')

In [215]:
df_sample_ = df_sample.rename({
    "origin_zone_id":"ORIG", 
    "destination_zone_id":"DEST",

}, axis=1)

df_sample_ = df_sample_[["ORIG", "DEST", "total_linked_trips", "total_unlinked_trips",'purpose_hbw', 'purpose_hbo', 'purpose_wbo', 'purpose_obo']]
df_sample_["TRIPS_HOMEBASED"] = df_sample_.purpose_hbw + df_sample_.purpose_hbo
df_sample_ = df_sample_[["ORIG", "DEST", "TRIPS_HOMEBASED"]]

df_sample_["ORIG"] = df_sample_["ORIG"].astype(str)
df_sample_["DEST"] = df_sample_["DEST"].astype(str)

In [216]:
# df_sample_["total_purp"] = df_sample_.purpose_hbw + df_sample_.purpose_hbo + df_sample_.purpose_wbo + df_sample_.purpose_obo
# df_sample_["is_link_match"] = df_sample_.total_linked_trips / df_sample_["total_purp"]
# df_sample_["is_unlink_match"] = df_sample_.total_unlinked_trips / df_sample_["total_purp"]
# df_sample_.describe()

In [217]:
df_sample_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3736797 entries, 0 to 3736796
Data columns (total 3 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   ORIG             object
 1   DEST             object
 2   TRIPS_HOMEBASED  int64 
dtypes: int64(1), object(2)
memory usage: 85.5+ MB


### potential sites

In [ ]:
# import random

# POTENTIAL_SITES_NUMBER = 150

# random.seed(41)
# t = random.sample(df_sample_.query("TRIPS_HOMEBASED > 1000").ORIG.unique().tolist(), POTENTIAL_SITES_NUMBER)
# P = set(t)

### bi-directional -> one directional

In [222]:
df_sample_

,ORIG,DEST,TRIPS_HOMEBASED
0,1005,130039602002,0
1,1005,130099702012,52
2,1005,130159601031,0
3,1005,130159602011,7
4,1005,130159604031,0
...,...,...,...
3736792,RWY3_WY,131850103014,0
3736793,RWY3_WY,132150004003,0
3736794,RWY3_WY,132679503005,0
3736795,RWY3_WY,132679504023,0


In [259]:
import random

POTENTIAL_SITES_NUMBER = 15

random.seed(41)
t = random.sample(df_sample_.query("TRIPS_HOMEBASED > 1000").ORIG.unique().tolist(), POTENTIAL_SITES_NUMBER)
P = set(t)
P = set(map(str, P))  # Ensure P is also strings

In [ ]:

"""
Optimized function to calculate trips dictionary for a subset of origins (P),
including only trips that either begin or end at P.

Args:
    df (pd.DataFrame): DataFrame with columns ["ORIG", "DEST", "TRIPS_HOMEBASED"].
    P (set): Subset of ORIG values.
    
Returns:
    dict: Dictionary with keys (P, ORIG) and values as the sum of P -> ORIG and ORIG -> P trips.
"""
df = df_sample_

# Filter the DataFrame to include only rows where ORIG or DEST is in P
df_filtered = df[(df["ORIG"].isin(P)) | (df["DEST"].isin(P))]

# Group by (ORIG, DEST) and sum TRIPS_HOMEBASED
grouped = df_filtered.groupby(["ORIG", "DEST"])["TRIPS_HOMEBASED"].sum()

# Convert grouped data to a dictionary for fast lookups
trip_lookup = grouped.to_dict()

# Initialize the trips dictionary
trips_dict = {}

# Calculate the sum of P -> ORIG and ORIG -> P for all pairs
for p in P:
    for orig in df_filtered["ORIG"].unique():
        # Get P -> ORIG
        p_to_orig = trip_lookup.get((p, orig), 0)
        
        # Get ORIG -> P
        orig_to_p = trip_lookup.get((orig, p), 0)
        
        # Update the dictionary with the sum
        trips_dict[(p, orig)] = p_to_orig + orig_to_p
    

# Keep only non-zero values in trips_dict
trips_dict = {key: value for key, value in trips_dict.items() if value != 0}


In [290]:
# trips_dict now is P -> A = P -> A + A -> P
trips_dict

{('130630405362', '130459105033'): 28,
 ('130630405362', '130510012002'): 21,
 ('130630405362', '130630402021'): 363,
 ('130630405362', '130630402022'): 68,
 ('130630405362', '130630402031'): 54,
 ('130630405362', '130630402032'): 23,
 ('130630405362', '130630402041'): 107,
 ('130630405362', '130630402042'): 230,
 ('130630405362', '130630403023'): 589,
 ('130630405362', '130630403024'): 23,
 ('130630405362', '130630403061'): 208,
 ('130630405362', '130630403062'): 162,
 ('130630405362', '130630403071'): 90,
 ('130630405362', '130630403081'): 27,
 ('130630405362', '130630403082'): 29,
 ('130630405362', '130630403083'): 63,
 ('130630405362', '130630404072'): 63,
 ('130630405362', '130630404091'): 34,
 ('130630405362', '130630404092'): 92,
 ('130630405362', '130630404094'): 15,
 ('130630405362', '130630404104'): 51,
 ('130630405362', '130630404141'): 164,
 ('130630405362', '130630404161'): 235,
 ('130630405362', '130630404162'): 399,
 ('130630405362', '130630404171'): 491,
 ('130630405362